In [1282]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats
import json
import re
from sklearn.preprocessing import StandardScaler
from IPython.core.interactiveshell import InteractiveShell
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
InteractiveShell.ast_node_interactivity = "all"
standard_X = StandardScaler()

In [1283]:
dfp = pd.read_csv('data-set/personal_valid.csv')
dfo = pd.read_csv('data-set/other_valid.csv')
dfp = dfp.rename(columns={'Unnamed: 0': 'ID'})
dfo = dfo.rename(columns={'Unnamed: 0': 'ID'})
pd.set_option('display.max_columns', 30)

## Integrácia dát a deduplikácia záznamov

Táto časť je skopírovaná z predchadzajúceho notebooku 'Prieskumná_analýza.ipynb'. Spojí dataset other a personal do jedného a následne odstráni duplikatne záznamy. 

In [1284]:
# merges duplicates
def merge(name):
    df = dfo_duplicates.loc[dfo_duplicates['name'] == name]
    return df.groupby(['name'], as_index=False).first()

def merge_other_personal(df):
    x = df.copy()

    dfo_duplicates = dfo[dfo.duplicated(['name'], keep=False)]
    dfo_unique = dfo.drop_duplicates(subset=["name"], keep=False)
    merged = []

    for name in dfo_duplicates['name'].unique():
        merged.append(merge(name))

    dfo_unique = dfo_unique.append(merged)

    x = pd.merge(dfp, dfo_unique, on='name')
    x = x.drop(columns=['address_y', 'ID_y'])
    x = x.rename(columns={"ID_x": "ID", "address_x": "address"})
    return x
    
df = merge_other_personal(df)

Dalsia vec ktoru musime spravit, je rozbalenie medical info. Nachadza sa tu: kurtosis_oxygen, mean_oxygen, skewness_oxygen a std_oxygen. Tieto hodnoty su ulozene ako object (string), pre to ich budeme musiet previest na cislo.

In [1285]:
def unpack_medical(df):
    x = df.copy()
    for i, row in x.iterrows():
        if not pd.isnull(x.at[i, 'medical_info']):
            x.at[i, 'medical_info'] = json.loads(x["medical_info"][i].replace("\'", "\""))
    # vytvorenie stlpcov z medical_info a ich spojenie so zvyskom dataframe
    df_med_info = x["medical_info"].apply(pd.Series)
    df_med_info = df_med_info.drop(0, 1)
    x = pd.concat([x, df_med_info], axis = 1).drop("medical_info", axis = 1)
    return x

df = unpack_medical(df)

Zmena kurtosis_oxygen, mean_oxygen, skewness_oxygen, std_oxygen z object na float

In [1286]:
def obj_to_float(df):
    x = df.copy()
    # kurtosis_oxygen
    x['kurtosis_oxygen'] = x['kurtosis_oxygen'].astype(np.float)
    # mean_oxygen
    x['mean_oxygen'] = x['mean_oxygen'].astype(np.float)
    # skewness_oxygen
    x['skewness_oxygen'] = x['skewness_oxygen'].astype(np.float)
    # std_oxygen
    x['std_oxygen'] = x['std_oxygen'].astype(np.float)
    return x
df = obj_to_float(df)

Rozhodli sme sa dropnut niektore stlpce. Meno, ID, fnlwgt. Dovodom je, ze kazda z tychto hodnot je rozna a na vyskyt cukrovky nema vplyv.  
Mame 2 udaje ktore nam ukazuju vek cloveka - age a date_of_birth. Kedze nemame ziadne nullove hodnoty pri age, nemusime podla datumu narodenia vek urcovat. Datum narodenia teda mozeme vyhodit

In [1287]:
def remove_unimportant_columns(df):
    x = df.copy()
    x = x.drop(['ID', 'name', 'fnlwgt', 'date_of_birth'], axis=1)
    return x
# df = remove_unimportant_columns(df)

Odstranenie space-ov z nazvov atributov

In [1288]:
def remove_space (df):
    x = df.copy()
    x = df.apply(lambda y: y.str.strip() if y.dtype == "object" else y)
    return x

df = remove_space(df)

Nahradenie hodnot, ktore mozu nadobudat len 2 stavy

In [1289]:
def put_0_1_values(df):
    # pohlavia
    x = df.copy()
    x['sex'] = x['sex'].replace('Male', 1)
    x['sex'] = x['sex'].replace('Female', 0)
    # tehotnost
    x['pregnant'] = x['pregnant'].replace(regex='(?i)f.*', value=0)
    x['pregnant'] = x['pregnant'].replace(regex='(?i)t.*', value=1)
    # muzi oznaceni ako tehotny su prepisani na 0
    x.loc[(x['pregnant'] == 1) & (x['sex'] == 1), 'pregnant'] = 0
    return x

df = put_0_1_values(df)

**Education a education-num**  
Zistujeme, co znamena education a education num. Predpoklad je, ze education-num je numericka reprezentacia education

In [1290]:
def education_analysis(df):
    # prints unique values in education
    x = df.copy()
    unique_edu = pd.unique(x['education'])
    print("Pred zjednotenim:\n", unique_edu)
    
    # Zjednotenie reprezentacii
    x['education'] = x['education'].replace(regex='(?i)_', value='-')
    unique_edu = pd.unique(x['education'])
    print("\nPo zjednoteni:\n", unique_edu)
    
    # hodnoty v education-num a v education
    print("\nHodnoty v jednotlivych education:")
    for item in unique_edu:
        edu_num = x.query("education == @item")["education-num"].unique()
        print(item, edu_num)

education_analysis(df)

Pred zjednotenim:
 ['Assoc-voc' 'Some-college' 'Assoc_voc' 'HS_grad' 'HS-grad' 'Bachelors'
 'Assoc-acdm' 'Doctorate' 'Some_college' '10th' '7th-8th' 'Masters' '11th'
 '12th' 'Prof-school' '1st-4th' '5th-6th' '9th' '1st_4th' '7th_8th'
 'Assoc_acdm' 'Prof_school' '5th_6th' 'Preschool']

Po zjednoteni:
 ['Assoc-voc' 'Some-college' 'HS-grad' 'Bachelors' 'Assoc-acdm' 'Doctorate'
 '10th' '7th-8th' 'Masters' '11th' '12th' 'Prof-school' '1st-4th'
 '5th-6th' '9th' 'Preschool']

Hodnoty v jednotlivych education:
Assoc-voc [ 1100.    11.    nan -1100.]
Some-college [   10.    nan  1000. -1000.]
HS-grad [   9.   nan  900. -900.]
Bachelors [   13. -1300.  1300.    nan]
Assoc-acdm [   12. -1200.    nan  1200.]
Doctorate [   16. -1600.    nan]
10th [   6.  600.   nan -600.]
7th-8th [   4.   nan -400.  400.]
Masters [   14.    nan -1400.  1400.]
11th [   7.  700. -700.   nan]
12th [800.  nan   8.]
Prof-school [15.]
1st-4th [ 2. nan]
5th-6th [  3. 300.]
9th [5.]
Preschool [1.]


Vidime ze education-num je ciselna reprezentacia education. Vytvorime si dictionary, ktory bude priradovat education ku education-num, s tym ze nechame celociselne reprezentacie od 1 po 16.

In [1291]:
#vrati unikatne hodnoty v stlpci education
def get_unique_edu(df):
    x = df.copy()
    x['education'] = x['education'].replace(regex='(?i)_', value='-')
    unique_edu = pd.unique(x['education'])
    return unique_edu

# rozne hodnoty education-num pre unikatny education zmeni na jedno (napr.: 5th-6th [  3. 300.] -> 3)
def get_edu_num(edu_num):
    for num in edu_num:
        if num is None:
            continue
        elif num < 100:
            return int(num)

def transform_education(df):
    x = df.copy()
    edu_to_num = {}
    #vytvorenie dictionary s moznymi hodnotami v jendotlivych education values
    for item in get_unique_edu(x):
        edu_num = x.query("education == @item")["education-num"].unique()
        edu_to_num[item] = get_edu_num(edu_num)
    
    # zmena moznych hodnot v education na rovnake
    x['education'] = x['education'].replace(regex='(?i)_', value='-')

    # namapuje nazvy education na cisla z dictionary
    x["education"] = x["education"].map(edu_to_num)
    
    # Dropne nepotrebny column education-num (bol nahradeny)
    x = x.drop(['education-num'], axis=1)
    return x
    
df = transform_education(df)

In [1292]:
def find_state(address):
    i = re.search('\x5cn.+\D', address)
    return address[i.start():i.end()][-3:-1]
    #return address[-8:][:2]

def address_to_state(df):
    x = df.copy()
    x['address'] = x['address'].apply(find_state)
    x = x.rename(columns={"address": "state"})
    return x

df = address_to_state(df)

,ID,name,state,age,sex,date_of_birth,race,marital-status,occupation,pregnant,relationship,skewness_glucose,mean_glucose,capital-gain,kurtosis_glucose,education,fnlwgt,class,std_glucose,income,native-country,hours-per-week,capital-loss,workclass,kurtosis_oxygen,mean_oxygen,skewness_oxygen,std_oxygen
0,0,Philip Miller,GA,48,1,1971-07-10 00 00 00,White,Married-civ-spouse,Other-service,0,Husband,0.400023,113.687500,0.0,0.038520,11,89040.0,0.0,45.326894,<=50K,United-States,40.0,0.0,Private,4.642782,7.227425,21.804034,31.430288
1,1,Mitch Wilson,KS,54,0,1965-08-09,White,Divorced,Machine-op-inspct,0,Own-child,31.250361,34.406250,0.0,5.243557,10,175370.0,1.0,31.877558,<=50K,United-States,40.0,0.0,Private,1.731936,41.427258,2.537562,61.235473
2,2,James Olsen,AE,48,0,1971-01-20 00:00:00,Black,Widowed,Machine-op-inspct,0,Unmarried,0.377293,101.328125,0.0,0.374884,10,212437.0,0.0,43.385465,<=50K,United-States,48.0,0.0,Private,7.134172,3.418896,58.953559,19.263642
3,3,Maurice Riley,AP,59,0,1959-11-02,White,Never-married,Prof-specialty,0,Not-in-family,35.459422,9.976562,0.0,5.859134,11,196584.0,1.0,32.258590,>50K,United-States,40.0,1564.0,Private,0.742921,68.934783,-0.089009,65.174611
4,4,Larry Stanley,MT,36,1,1983-08-13,White,Married-civ-spouse,Handlers-cleaners,0,Husband,-0.243264,134.835938,0.0,-0.299902,9,92036.0,0.0,50.723635,>50K,United-States,40.0,0.0,Private,9.078860,2.269231,89.748456,18.351895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1306,1306,Michael Everett,AL,46,0,1972-11-13,White,Divorced,Prof-specialty,0,Unmarried,2.611897,85.828125,0.0,0.783824,13,177599.0,0.0,35.656589,<=50K,United-States,35.0,0.0,Local-gov,7.701320,3.136288,73.019918,17.460629
1307,1307,Julian Patrick,TN,53,1,1965-12-04,White,Never-married,Handlers-cleaners,0,Own-child,34.021146,35.929688,0.0,5.326209,9,162667.0,1.0,31.317723,<=50K,Portugal,50.0,0.0,Private,2.669580,22.634615,6.700531,50.468320
1308,1308,Lee Walker,RI,33,1,1986-09-20,White,Married-civ-spouse,Sales,0,Husband,-0.515704,125.968750,0.0,0.106988,9,106961.0,0.0,51.607130,>50K,United-States,40.0,0.0,Private,9.371372,2.594482,104.136934,16.903779
1309,1309,Thomas Garrison,LA,55,1,64-05-10,White,Never-married,Craft-repair,0,Own-child,0.303495,127.250000,0.0,0.111974,7,313873.0,1.0,45.399098,<=50K,United-States,30.0,0.0,Private,5.559185,6.393813,30.623632,33.970594


Income vieme zmenit na 2 hodnoty: <=50K = 0; >50K = 1. Mame jednu hodnotu NaN a nevieme ju predikovat.

In [1095]:
df['income'].unique()
df_null = df[df['income'].isnull()]
df_null
df_tetst = df.loc[df['occupation'] == 'Sales']
df_tetst = df_tetst.loc[df_tetst['hours-per-week'] > 45.0]
df_tetst[['age', 'sex', 'race', 'marital-status', 'occupation', 'income', 'hours-per-week', 'education', 'workclass', 'relationship']]

array(['<=50K', '>50K', nan], dtype=object)

,ID,name,address,age,sex,date_of_birth,race,marital-status,occupation,pregnant,...,std_glucose,income,native-country,hours-per-week,capital-loss,workclass,kurtosis_oxygen,mean_oxygen,skewness_oxygen,std_oxygen
593,593,Carl Nelson,"830 Melinda Harbor\r\nFrosttown, NV 26060",67,Female,1952-09-06 00 00 00,White,Divorced,Sales,F,...,34.645251,NaN,United-States,48.0,0.0,Private,11.514697,1.843645,168.109415,12.963409


,age,sex,race,marital-status,occupation,income,hours-per-week,education,workclass,relationship
56,53,Male,Asian-Pac-Islander,Never-married,Sales,<=50K,60.0,Bachelors,self-emp-not-inc,Own-child
136,44,Female,White,Married-civ-spouse,Sales,<=50K,60.0,HS-grad,Self-emp-inc,Wife
157,52,Male,White,Never-married,Sales,>50K,50.0,Masters,Private,Not-in-family
172,55,Male,White,Married-civ-spouse,Sales,<=50K,50.0,11th,private,Husband
225,58,Male,White,Married-civ-spouse,Sales,<=50K,60.0,Assoc-voc,Private,Husband
318,38,Female,White,??,Sales,<=50K,47.0,Some-college,Private,Own-child
375,59,Male,White,Married-civ-spouse,Sales,>50K,50.0,HS-grad,Private,Husband
400,40,Male,White,Married-civ-spouse,Sales,<=50K,55.0,Some-college,Private,Husband
456,60,Female,White,Divorced,Sales,<=50K,55.0,Some-college,Private,Unmarried
471,56,Female,White,Married-civ-spouse,Sales,>50K,52.0,Some-college,Private,Wife


In [1096]:
df['workclass'].unique()

array(['Private', 'private', 'Local-gov', '?', 'Self-emp-not-inc',
       'State-gov', 'Federal-gov', 'self-emp-inc', 'self-emp-not-inc',
       'Self-emp-inc', 'state-gov', nan, 'local-gov', 'federal-gov'],
      dtype=object)

In [1097]:
df.loc[df['native-country'] == '?'][['address', 'native-country']]

,address,native-country
73,"25535 Bird Rue\r\nHarrisfort, NJ 71162",?
105,"159 French Ways\r\nWest Paul, CT 44122",?
191,"7115 Diana Heights\r\nVictorside, MT 47375",?
255,"851 Burns Isle Suite 672\r\nLake Ericchester, ...",?
311,"56110 Blake Point\r\nKatiefort, NY 02127",?
322,"371 Steven Station Suite 485\r\nPaulchester, M...",?
420,"1636 Kline Street Apt. 137\r\nHesterport, VT 9...",?
436,"534 William Road Suite 795\r\nWest Joseph, VT ...",?
567,"2125 Joseph Vista Apt. 972\r\nRiverafort, VA 5...",?
735,67806 Brown Forest Apt. 872\r\nNorth Stacybury...,?


In [1100]:
pipeline = Pipeline([
    ('test', FunctionTransformer(
        func=put_0_1_values                        # function to be used
        )) # parameters to the function
])
df = pipeline.fit_transform(df)
df

,ID,name,address,age,sex,race,marital-status,occupation,pregnant,education-num,...,std_glucose,income,native-country,hours-per-week,capital-loss,workclass,kurtosis_oxygen,mean_oxygen,skewness_oxygen,std_oxygen
0,0,Philip Miller,"7910 Rosales Plain Apt. 454\r\nPort Carl, GA 6...",48,1,White,Married-civ-spouse,Other-service,0,1100.0,...,45.326894,<=50K,United-States,40.0,0.0,Private,4.642782,7.227425,21.804034,31.430288
1,1,Mitch Wilson,055 Morgan Plains Suite 225\r\nEast Darrylmout...,54,0,White,Divorced,Machine-op-inspct,0,10.0,...,31.877558,<=50K,United-States,40.0,0.0,Private,1.731936,41.427258,2.537562,61.235473
2,2,James Olsen,"PSC 7359, Box 2088\r\nAPO AE 62717",48,0,Black,Widowed,Machine-op-inspct,0,10.0,...,43.385465,<=50K,United-States,48.0,0.0,Private,7.134172,3.418896,58.953559,19.263642
3,3,Maurice Riley,USS Allen\r\nFPO AP 87677,59,0,White,Never-married,Prof-specialty,0,11.0,...,32.258590,>50K,United-States,40.0,1564.0,Private,0.742921,68.934783,-0.089009,65.174611
4,4,Larry Stanley,"491 Fields Key Suite 544\r\nGracestad, MT 13456",36,1,White,Married-civ-spouse,Handlers-cleaners,0,9.0,...,50.723635,>50K,United-States,40.0,0.0,Private,9.078860,2.269231,89.748456,18.351895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1306,1306,Michael Everett,"35892 Robert Forest Apt. 556\r\nRubioside, AL ...",46,0,White,Divorced,Prof-specialty,0,NaN,...,35.656589,<=50K,United-States,35.0,0.0,Local-gov,7.701320,3.136288,73.019918,17.460629
1307,1307,Julian Patrick,"53981 Taylor Brook\r\nBarkerville, TN 17001",53,1,White,Never-married,Handlers-cleaners,0,9.0,...,31.317723,<=50K,Portugal,50.0,0.0,Private,2.669580,22.634615,6.700531,50.468320
1308,1308,Lee Walker,"847 Jamie Cove Apt. 464\r\nCummingsview, RI 90999",33,1,White,Married-civ-spouse,Sales,0,9.0,...,51.607130,>50K,United-States,40.0,0.0,Private,9.371372,2.594482,104.136934,16.903779
1309,1309,Thomas Garrison,"159 Yvonne Way Suite 696\r\nWest Ashleyton, LA...",55,1,White,Never-married,Craft-repair,0,NaN,...,45.399098,<=50K,United-States,30.0,0.0,Private,5.559185,6.393813,30.623632,33.970594
